# Import Modules

In [55]:
import pandas as pd
import numpy as np

# Load Scraped Data

In [56]:
races_winner = pd.read_csv("../data/raw/races_winner.csv", encoding="utf-8")
races_results = pd.read_csv("../data/intermediate/races_results.csv", encoding="utf-8")
races_fastest_laps = pd.read_csv("../data/raw/races_fastest_laps.csv", encoding="utf-8")
races_pits = pd.read_csv("../data/raw/races_pits.csv", encoding="utf-8")
races_start = pd.read_csv("../data/raw/races_start.csv", encoding="utf-8")
drivers_info = pd.read_csv("../data/raw/drivers_info.csv", encoding="utf-8")

# Prepare Data for Merging

### Fix Start/Finish Missing Data

In [57]:
print(races_results.shape) 
print(races_start.shape) 

(24963, 9)
(25018, 5)


In [58]:
# Merge with indicator to track presence
merged = pd.merge(
    races_start,
    races_results,
    on=["race_url", "driver"],
    how="left",
    indicator=True
)

missing_finishers = merged[merged["_merge"] == "left_only"]

display(missing_finishers[["race_url", "driver", "team_name_x", "start_position"]]) # Show drivers that started but didn't finish


,race_url,driver,team_name_x,start_position
38,https://www.formula1.com/en/results/1950/races...,Alfredo Pián,Maserati,18
152,https://www.formula1.com/en/results/1950/races...,Felice Bonetto,Maserati,23
378,https://www.formula1.com/en/results/1951/races...,Reg Parnell,BRM,8
380,https://www.formula1.com/en/results/1951/races...,Ken Richardson,BRM,10
410,https://www.formula1.com/en/results/1951/races...,Juan Jover,Maserati,18
...,...,...,...,...
17907,https://www.formula1.com/en/results/2007/races...,Felipe Massa,Ferrari,5
17911,https://www.formula1.com/en/results/2007/races...,Giancarlo Fisichella,Renault,9
18117,https://www.formula1.com/en/results/2007/races...,Alexander Wurz,Williams Toyota,18
18175,https://www.formula1.com/en/results/2008/races...,Rubens Barrichello,Honda,10


In [59]:
extra_finishers = pd.merge(
    races_results,
    races_start,
    on=["race_url", "driver"],
    how="left",
    indicator=True
)

extra_finishers = extra_finishers[extra_finishers["_merge"] == "left_only"]

display(extra_finishers[["race_url", "driver", "team_name_x", "raw_end_position"]]) #  Show drivers that finished but didn't start

,race_url,driver,team_name_x,raw_end_position
10,https://www.formula1.com/en/results/1950/races...,Brian Shawe-Taylor,Maserati,10
21,https://www.formula1.com/en/results/1950/races...,Tony Rolt,ERA,NC
71,https://www.formula1.com/en/results/1950/races...,Bill Cantrell,Adams Offenhauser,27
181,https://www.formula1.com/en/results/1951/races...,Manny Ayulo,Kurtis Kraft Offenhauser,3
500,https://www.formula1.com/en/results/1952/races...,Peter Hirt,Ferrari,11
...,...,...,...,...
23937,https://www.formula1.com/en/results/2022/races...,Valtteri Bottas,Alfa Romeo Ferrari,11
24084,https://www.formula1.com/en/results/2022/races...,Pierre Gasly,AlphaTauri RBPT,18
24211,https://www.formula1.com/en/results/2023/races...,Sergio Perez,Red Bull Racing Honda RBPT,5
24217,https://www.formula1.com/en/results/2023/races...,Valtteri Bottas,Alfa Romeo Ferrari,11


In [60]:
# Insert missing finishers
for _, row in missing_finishers.iterrows():
    races_results = pd.concat([
        races_results,
        pd.DataFrame([{
            "race_url": row["race_url"],
            "raw_end_position": "DNF",
            "driver": row["driver"],
            "team_name": row["team_name_x"],
            "laps_completed": "0",
            "time_gap": "N/A",
            "points": "0",
            "end_position": "LAST",
            "position_status": "DNF"
        }])
    ], ignore_index=True)

# Group by race_url and get max numeric position for each race
max_positions = races_results.groupby("race_url").agg(max_pos=("end_position", lambda x: pd.to_numeric(x, errors="coerce").max())).reset_index()

# Merge max positions back to main dataframe
races_results = races_results.merge(max_positions, on="race_url", how="left")

# Replace LAST with max_pos + 1 
races_results.loc[races_results["end_position"] == "LAST", "end_position"] = (races_results["max_pos"] + 1).astype(str)

# Drop the temporary max_pos column
races_results = races_results.drop("max_pos", axis=1)

# Insert missing starters
for _, row in extra_finishers.iterrows():
    races_start = pd.concat([
        races_start,
        pd.DataFrame([{
            "start_position": "LAST",
            "driver": row["driver"],
            "team_name": row["team_name_x"],
            "qualifying_time": "Unknown",
            "race_url": row["race_url"]
        }])
    ], ignore_index=True)

# Group by race_url and get max numeric position for each race
max_positions = races_start.groupby("race_url").agg(max_pos=("start_position", lambda x: pd.to_numeric(x, errors="coerce").max())).reset_index()

# Merge max positions back to main dataframe
races_start = races_start.merge(max_positions, on="race_url", how="left")

# Replace LAST with max_pos + 1 
races_start.loc[races_start["start_position"] == "LAST", "start_position"] = (races_start["max_pos"] + 1).astype(str)

# Drop the temporary max_pos column
races_start = races_start.drop("max_pos", axis=1)

In [61]:
print(races_results.shape) # 25256
print(races_start.shape) # 25162

(25256, 9)
(25162, 5)


In [62]:
# Duplicated rows
print(races_results.duplicated(subset=["race_url", "driver"]).sum()) # 155
print(races_start.duplicated(subset=["race_url", "driver"]).sum()) # 61

155
61


In [63]:
# Some racers have multiple rows for some reason. Some finish 11 but also have "NC". This code keeps the best result (ie 11th) and removes the other

# Resolve duplicates for the races_results dataframe
def resolve_duplicates(group):
    if len(group) == 1:
        return group

    # Filter for valid finishes
    valid_positions = group[~group["end_position"].isin(["DNF", "NC", "DQ", "DNS", "EX"])]
    if not valid_positions.empty:
        return valid_positions.head(1)  # Keep first valid finish

    # If all are non-finishes, just keep the first one
    return group.head(1)

dup_drivers = races_results[races_results.duplicated(subset=["race_url", "driver"], keep=False)]
before_shape = races_results.shape
cleaned_results = dup_drivers.groupby(["race_url", "driver"]).apply(resolve_duplicates).reset_index(drop=True)

# Drop the duplicates from original and replace with cleaned
races_results = pd.concat([
    races_results.drop(dup_drivers.index),
    cleaned_results
], ignore_index=True)
after_shape = races_results.shape
print(f"Dropped {before_shape[0] - after_shape[0]} rows from races_results dataframe") # 155 rows


# Resolve duplicates for the races_start dataframe
def resolve_start_duplicates(group):
    if len(group) == 1:
        return group
    
    # Prefer the row with a numeric start_position (ie not "Unknown")
    numeric_starts = group[group["start_position"].apply(lambda x: str(x).isdigit())]
    
    if not numeric_starts.empty:
        return numeric_starts.head(1)
    
    # If all start_positions are "Unknown" or non-numeric, just take the first
    return group.head(1)

dup_starts = races_start[races_start.duplicated(subset=["race_url", "driver"], keep=False)]
before_shape = races_start.shape
cleaned_starts = dup_starts.groupby(["race_url", "driver"]).apply(resolve_start_duplicates).reset_index(drop=True)

# Drop the duplicates from original and replace with cleaned
races_start = pd.concat([
    races_start.drop(dup_starts.index),
    cleaned_starts
], ignore_index=True)
after_shape = races_start.shape
print(f"Dropped {before_shape[0] - after_shape[0]} rows from races_starts dataframe") # 61 rows

Dropped 155 rows from races_results dataframe
Dropped 61 rows from races_starts dataframe


C:\Users\jackw\AppData\Local\Temp\ipykernel_13788\2569926271.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cleaned_results = dup_drivers.groupby(["race_url", "driver"]).apply(resolve_duplicates).reset_index(drop=True)
C:\Users\jackw\AppData\Local\Temp\ipykernel_13788\2569926271.py:45: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cleaned_starts = dup_starts.groupby(["race_url", "driver"]).apply(resol

In [64]:
print(races_results.shape)
print(races_start.shape)
# Have equal rows now 25101

(25101, 9)
(25101, 5)


In [65]:
# Duplicated rows (0)
print(races_results.duplicated(subset=["race_url", "driver"]).sum())
print(races_start.duplicated(subset=["race_url", "driver"]).sum())

0
0


### Fix Race Winner Duplicates

In [66]:
display(races_winner[races_winner["race_url"].duplicated(keep=False)]) # These are shared car race winners

,year,race,round,date,winner,team_name,laps,total_time,race_url
10,1951,France,4,01 Jul 1951,Juan Manuel Fangio,Alfa Romeo,77.0,3:22:11.000,https://www.formula1.com/en/results/1951/races...
11,1951,France,5,01 Jul 1951,Luigi Fagioli,Alfa Romeo,NaN,NaN,https://www.formula1.com/en/results/1951/races...
49,1956,Argentina,1,22 Jan 1956,Luigi Musso,Ferrari,NaN,NaN,https://www.formula1.com/en/results/1956/races...
50,1956,Argentina,2,22 Jan 1956,Juan Manuel Fangio,Ferrari,98.0,3:00:03.700,https://www.formula1.com/en/results/1956/races...
62,1957,Great Britain,5,20 Jul 1957,Tony Brooks,Vanwall,NaN,NaN,https://www.formula1.com/en/results/1957/races...
63,1957,Great Britain,6,20 Jul 1957,Stirling Moss,Vanwall,90.0,3:06:37.800,https://www.formula1.com/en/results/1957/races...


In [67]:
# Update shared car race winners to include both drivers
races_winner.loc[10, "winner"] = "Juan Manuel Fangio / Luigi Fagioli"
races_winner.loc[50, "winner"] = "Juan Manuel Fangio / Luigi Musso" 
races_winner.loc[63, "winner"] = "Stirling Moss / Tony Brooks"

In [68]:
index_to_drop = [11, 49, 62]
races_winner.drop(index_to_drop, inplace=True)

In [69]:
races_winner.loc[races_winner["year"] == 1951]

,year,race,round,date,winner,team_name,laps,total_time,race_url
7,1951,Switzerland,1,27 May 1951,Juan Manuel Fangio,Alfa Romeo,42.0,2:07:53.640,https://www.formula1.com/en/results/1951/races...
8,1951,Indianapolis,2,30 May 1951,Lee Wallard,Kurtis Kraft Offenhauser,200.0,3:57:38.050,https://www.formula1.com/en/results/1951/races...
9,1951,Belgium,3,17 Jun 1951,Nino Farina,Alfa Romeo,36.0,2:45:46.200,https://www.formula1.com/en/results/1951/races...
10,1951,France,4,01 Jul 1951,Juan Manuel Fangio / Luigi Fagioli,Alfa Romeo,77.0,3:22:11.000,https://www.formula1.com/en/results/1951/races...
12,1951,Great Britain,6,14 Jul 1951,Jose Froilan Gonzalez,Ferrari,90.0,2:42:18.200,https://www.formula1.com/en/results/1951/races...
13,1951,Germany,7,29 Jul 1951,Alberto Ascari,Ferrari,20.0,3:23:03.300,https://www.formula1.com/en/results/1951/races...
14,1951,Italy,8,16 Sep 1951,Alberto Ascari,Ferrari,80.0,2:42:39.300,https://www.formula1.com/en/results/1951/races...
15,1951,Spain,9,28 Oct 1951,Juan Manuel Fangio,Alfa Romeo,70.0,2:46:54.100,https://www.formula1.com/en/results/1951/races...


In [70]:
races_winner.loc[12:15, "round"] = races_winner.loc[12:15, "round"] - 1 # Fix round numbers for 1951
races_winner.loc[50:57, "round"] = races_winner.loc[50:57, "round"] - 1 # Fix round numbers for 1956
races_winner.loc[63:66, "round"] = races_winner.loc[63:66, "round"] - 1 # Fix round numbers for 1957

In [71]:
races_winner.loc[races_winner["year"] == 1951]

,year,race,round,date,winner,team_name,laps,total_time,race_url
7,1951,Switzerland,1,27 May 1951,Juan Manuel Fangio,Alfa Romeo,42.0,2:07:53.640,https://www.formula1.com/en/results/1951/races...
8,1951,Indianapolis,2,30 May 1951,Lee Wallard,Kurtis Kraft Offenhauser,200.0,3:57:38.050,https://www.formula1.com/en/results/1951/races...
9,1951,Belgium,3,17 Jun 1951,Nino Farina,Alfa Romeo,36.0,2:45:46.200,https://www.formula1.com/en/results/1951/races...
10,1951,France,4,01 Jul 1951,Juan Manuel Fangio / Luigi Fagioli,Alfa Romeo,77.0,3:22:11.000,https://www.formula1.com/en/results/1951/races...
12,1951,Great Britain,5,14 Jul 1951,Jose Froilan Gonzalez,Ferrari,90.0,2:42:18.200,https://www.formula1.com/en/results/1951/races...
13,1951,Germany,6,29 Jul 1951,Alberto Ascari,Ferrari,20.0,3:23:03.300,https://www.formula1.com/en/results/1951/races...
14,1951,Italy,7,16 Sep 1951,Alberto Ascari,Ferrari,80.0,2:42:39.300,https://www.formula1.com/en/results/1951/races...
15,1951,Spain,8,28 Oct 1951,Juan Manuel Fangio,Alfa Romeo,70.0,2:46:54.100,https://www.formula1.com/en/results/1951/races...


### Fix Fastest Lap Duplicates

In [72]:
display(races_fastest_laps[races_fastest_laps["race_url"].duplicated(keep=False)]) # These are shared car races so I will just keep the ones with data

,race_url,driver,fastest_lap
10,https://www.formula1.com/en/results/1951/races...,Juan Manuel Fangio,1
11,https://www.formula1.com/en/results/1951/races...,Juan Manuel Fangio,1
49,https://www.formula1.com/en/results/1956/races...,Juan Manuel Fangio,1
50,https://www.formula1.com/en/results/1956/races...,Juan Manuel Fangio,1
62,https://www.formula1.com/en/results/1957/races...,Stirling Moss,1
63,https://www.formula1.com/en/results/1957/races...,Stirling Moss,1


In [73]:
races_fastest_laps.shape

(1125, 3)

In [74]:
races_fastest_laps.drop_duplicates(inplace=True)

In [75]:
races_fastest_laps.shape # Good. Only dropped the 3

(1122, 3)

### Fix Fastest Lap Missing Data

In [76]:
missing_urls = set(races_fastest_laps["race_url"].unique()) - set(races_winner["race_url"].unique())
print("Missing race_urls from df2:", missing_urls)

missing_urls = set(races_winner["race_url"].unique()) - set(races_fastest_laps["race_url"].unique())
print("Missing race_urls from df1:", missing_urls)
# 3 missing entries from races_fastest_laps - 2021 belgium was suspended shortly after start so couldn't find any fastest lap info

Missing race_urls from df2: set()
Missing race_urls from df1: {'https://www.formula1.com/en/results/2023/races/1217/netherlands/race-result', 'https://www.formula1.com/en/results/2023/races/1210/monaco/race-result', 'https://www.formula1.com/en/results/2021/races/1074/belgium/race-result'}


In [77]:
new_data = [
    {"race_url": "https://www.formula1.com/en/results/2023/races/1217/netherlands/race-result", "driver": "Fernando Alonso", "fastest_lap": 1},
    {"race_url": "https://www.formula1.com/en/results/2023/races/1210/monaco/race-result", "driver": "Lewis Hamilton", "fastest_lap": 1}
    ]

# Add new data to dataframe
new_rows = pd.DataFrame(new_data)
races_fastest_laps = pd.concat([races_fastest_laps, new_rows], ignore_index=True)

# Check to make sure theres only 1 url missing (belgium 2021)
missing_urls = set(races_winner["race_url"].unique()) - set(races_fastest_laps["race_url"].unique())
print("Missing race_urls from df1:", missing_urls)

Missing race_urls from df1: {'https://www.formula1.com/en/results/2021/races/1074/belgium/race-result'}


# Merge Dataframes

### Display all Dataframes

In [78]:
display(races_results)
display(races_start)
display(races_winner)
display(races_fastest_laps)
display(races_pits)

,race_url,raw_end_position,driver,team_name,laps_completed,time_gap,points,end_position,position_status
0,https://www.formula1.com/en/results/1950/races...,1,Nino Farina,Alfa Romeo,70.0,2:13:23.600,9.0,1,CLAS
1,https://www.formula1.com/en/results/1950/races...,2,Luigi Fagioli,Alfa Romeo,70.0,+2.600s,6.0,2,CLAS
2,https://www.formula1.com/en/results/1950/races...,3,Reg Parnell,Alfa Romeo,70.0,+52.000s,4.0,3,CLAS
3,https://www.formula1.com/en/results/1950/races...,4,Yves Giraud-Cabantous,Talbot-Lago,68.0,+2 laps,3.0,4,CLAS
4,https://www.formula1.com/en/results/1950/races...,5,Louis Rosier,Talbot-Lago,68.0,+2 laps,2.0,5,CLAS
...,...,...,...,...,...,...,...,...,...
25096,https://www.formula1.com/en/results/1961/races...,DQ,Stirling Moss,Ferguson Climax,56.0,DSQ,0.0,19,DQ
25097,https://www.formula1.com/en/results/1961/races...,11,Masten Gregory,Lotus Climax,92.0,+8 laps,0.0,11,CLAS
25098,https://www.formula1.com/en/results/1964/races...,7,Jim Clark,Lotus Climax,102.0,DNF,0.0,7,CLAS
25099,https://www.formula1.com/en/results/1964/races...,7,Mike Spence,Lotus Climax,NaN,SHC,0.0,7,CLAS


,start_position,driver,team_name,qualifying_time,race_url
0,1,Nino Farina,Alfa Romeo,1:50.800,https://www.formula1.com/en/results/1950/races...
1,2,Luigi Fagioli,Alfa Romeo,1:51.000,https://www.formula1.com/en/results/1950/races...
2,3,Juan Manuel Fangio,Alfa Romeo,1:51.000,https://www.formula1.com/en/results/1950/races...
3,4,Reg Parnell,Alfa Romeo,1:52.200,https://www.formula1.com/en/results/1950/races...
4,5,Prince Bira,Maserati,1:52.600,https://www.formula1.com/en/results/1950/races...
...,...,...,...,...,...
25096,15,Roy Salvadori,Cooper Climax,2:07.400,https://www.formula1.com/en/results/1957/races...
25097,1,Stirling Moss,Vanwall,2:00.200,https://www.formula1.com/en/results/1957/races...
25098,6,Stuart Lewis-Evans,Vanwall,2:01.200,https://www.formula1.com/en/results/1957/races...
25099,3,Tony Brooks,Vanwall,2:00.400,https://www.formula1.com/en/results/1957/races...


,year,race,round,date,winner,team_name,laps,total_time,race_url
0,1950,Great Britain,1,13 May 1950,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...
1,1950,Monaco,2,21 May 1950,Juan Manuel Fangio,Alfa Romeo,100.0,3:13:18.700,https://www.formula1.com/en/results/1950/races...
2,1950,Indianapolis,3,30 May 1950,Johnnie Parsons,Kurtis Kraft Offenhauser,138.0,2:46:55.970,https://www.formula1.com/en/results/1950/races...
3,1950,Switzerland,4,04 Jun 1950,Nino Farina,Alfa Romeo,42.0,2:02:53.700,https://www.formula1.com/en/results/1950/races...
4,1950,Belgium,5,18 Jun 1950,Juan Manuel Fangio,Alfa Romeo,35.0,2:47:26.000,https://www.formula1.com/en/results/1950/races...
...,...,...,...,...,...,...,...,...,...
1123,2024,Mexico,20,27 Oct 2024,Carlos Sainz,Ferrari,71.0,1:40:55.800,https://www.formula1.com/en/results/2024/races...
1124,2024,Brazil,21,03 Nov 2024,Max Verstappen,Red Bull Racing Honda RBPT,69.0,2:06:54.430,https://www.formula1.com/en/results/2024/races...
1125,2024,Las Vegas,22,23 Nov 2024,George Russell,Mercedes,50.0,1:22:05.969,https://www.formula1.com/en/results/2024/races...
1126,2024,Qatar,23,01 Dec 2024,Max Verstappen,Red Bull Racing Honda RBPT,57.0,1:31:05.323,https://www.formula1.com/en/results/2024/races...


,race_url,driver,fastest_lap
0,https://www.formula1.com/en/results/1950/races...,Nino Farina,1
1,https://www.formula1.com/en/results/1950/races...,Juan Manuel Fangio,1
2,https://www.formula1.com/en/results/1950/races...,Johnnie Parsons,1
3,https://www.formula1.com/en/results/1950/races...,Nino Farina,1
4,https://www.formula1.com/en/results/1950/races...,Nino Farina,1
...,...,...,...
1119,https://www.formula1.com/en/results/2024/races...,Lando Norris,1
1120,https://www.formula1.com/en/results/2024/races...,Lando Norris,1
1121,https://www.formula1.com/en/results/2024/races...,Kevin Magnussen,1
1122,https://www.formula1.com/en/results/2023/races...,Fernando Alonso,1


,pit_count,driver,race_url
0,1,Nelson Piquet,https://www.formula1.com/en/results/1983/races...
1,1,Keke Rosberg,https://www.formula1.com/en/results/1983/races...
2,1,Eddie Cheever,https://www.formula1.com/en/results/1983/races...
3,1,Alain Prost,https://www.formula1.com/en/results/1983/races...
4,1,Nelson Piquet,https://www.formula1.com/en/results/1983/races...
...,...,...,...
10804,1,Carlos Sainz,https://www.formula1.com/en/results/2024/races...
10805,1,Lando Norris,https://www.formula1.com/en/results/2024/races...
10806,1,George Russell,https://www.formula1.com/en/results/2024/races...
10807,1,Max Verstappen,https://www.formula1.com/en/results/2024/races...


### Merge races_results -- races_start on [race_url, driver]

In [79]:
races_all_results = pd.merge(races_results, races_start, on=["race_url", "driver"], how="inner")
races_all_results.shape # Still has 25101

(25101, 12)

In [80]:
# Drop team_name_y column
races_all_results = races_all_results.drop(columns=["team_name_y"])

In [81]:
races_all_results.head()

,race_url,raw_end_position,driver,team_name_x,laps_completed,time_gap,points,end_position,position_status,start_position,qualifying_time
0,https://www.formula1.com/en/results/1950/races...,1,Nino Farina,Alfa Romeo,70.0,2:13:23.600,9.0,1,CLAS,1,1:50.800
1,https://www.formula1.com/en/results/1950/races...,2,Luigi Fagioli,Alfa Romeo,70.0,+2.600s,6.0,2,CLAS,2,1:51.000
2,https://www.formula1.com/en/results/1950/races...,3,Reg Parnell,Alfa Romeo,70.0,+52.000s,4.0,3,CLAS,4,1:52.200
3,https://www.formula1.com/en/results/1950/races...,4,Yves Giraud-Cabantous,Talbot-Lago,68.0,+2 laps,3.0,4,CLAS,6,1:53.400
4,https://www.formula1.com/en/results/1950/races...,5,Louis Rosier,Talbot-Lago,68.0,+2 laps,2.0,5,CLAS,9,1:56.000


### Merge races_all_results -- races_pits on [race_url, driver]

In [82]:
races_all_results_pits = pd.merge(races_all_results, races_pits, on=["race_url", "driver"], how="left")
races_all_results_pits.shape # Still has 25101 rows

(25101, 12)

In [83]:
races_all_results_pits.head()

,race_url,raw_end_position,driver,team_name_x,laps_completed,time_gap,points,end_position,position_status,start_position,qualifying_time,pit_count
0,https://www.formula1.com/en/results/1950/races...,1,Nino Farina,Alfa Romeo,70.0,2:13:23.600,9.0,1,CLAS,1,1:50.800,NaN
1,https://www.formula1.com/en/results/1950/races...,2,Luigi Fagioli,Alfa Romeo,70.0,+2.600s,6.0,2,CLAS,2,1:51.000,NaN
2,https://www.formula1.com/en/results/1950/races...,3,Reg Parnell,Alfa Romeo,70.0,+52.000s,4.0,3,CLAS,4,1:52.200,NaN
3,https://www.formula1.com/en/results/1950/races...,4,Yves Giraud-Cabantous,Talbot-Lago,68.0,+2 laps,3.0,4,CLAS,6,1:53.400,NaN
4,https://www.formula1.com/en/results/1950/races...,5,Louis Rosier,Talbot-Lago,68.0,+2 laps,2.0,5,CLAS,9,1:56.000,NaN


### Merge races_all_results_pits -- races_winner on [race_url]

In [84]:
races_all_results_pits_winner = pd.merge(races_all_results_pits, races_winner, on=["race_url"], how="left")
races_all_results_pits_winner.shape # Still has 25101 rows

(25101, 20)

In [85]:
races_all_results_pits_winner.head()

,race_url,raw_end_position,driver,team_name_x,laps_completed,time_gap,points,end_position,position_status,start_position,qualifying_time,pit_count,year,race,round,date,winner,team_name,laps,total_time
0,https://www.formula1.com/en/results/1950/races...,1,Nino Farina,Alfa Romeo,70.0,2:13:23.600,9.0,1,CLAS,1,1:50.800,NaN,1950,Great Britain,1,13 May 1950,Nino Farina,Alfa Romeo,70.0,2:13:23.600
1,https://www.formula1.com/en/results/1950/races...,2,Luigi Fagioli,Alfa Romeo,70.0,+2.600s,6.0,2,CLAS,2,1:51.000,NaN,1950,Great Britain,1,13 May 1950,Nino Farina,Alfa Romeo,70.0,2:13:23.600
2,https://www.formula1.com/en/results/1950/races...,3,Reg Parnell,Alfa Romeo,70.0,+52.000s,4.0,3,CLAS,4,1:52.200,NaN,1950,Great Britain,1,13 May 1950,Nino Farina,Alfa Romeo,70.0,2:13:23.600
3,https://www.formula1.com/en/results/1950/races...,4,Yves Giraud-Cabantous,Talbot-Lago,68.0,+2 laps,3.0,4,CLAS,6,1:53.400,NaN,1950,Great Britain,1,13 May 1950,Nino Farina,Alfa Romeo,70.0,2:13:23.600
4,https://www.formula1.com/en/results/1950/races...,5,Louis Rosier,Talbot-Lago,68.0,+2 laps,2.0,5,CLAS,9,1:56.000,NaN,1950,Great Britain,1,13 May 1950,Nino Farina,Alfa Romeo,70.0,2:13:23.600


### Merge races_all_results_pits_winner -- races_fastest_laps on [race_url, driver]

In [86]:
races_all_results_pits_winner_fl = pd.merge(races_all_results_pits_winner, races_fastest_laps, on=["race_url", "driver"], how="left")
races_all_results_pits_winner_fl.shape # Still has 25101 rows

(25101, 21)

In [87]:
races_all_results_pits_winner_fl.head()

,race_url,raw_end_position,driver,team_name_x,laps_completed,time_gap,points,end_position,position_status,start_position,...,pit_count,year,race,round,date,winner,team_name,laps,total_time,fastest_lap
0,https://www.formula1.com/en/results/1950/races...,1,Nino Farina,Alfa Romeo,70.0,2:13:23.600,9.0,1,CLAS,1,...,NaN,1950,Great Britain,1,13 May 1950,Nino Farina,Alfa Romeo,70.0,2:13:23.600,1.0
1,https://www.formula1.com/en/results/1950/races...,2,Luigi Fagioli,Alfa Romeo,70.0,+2.600s,6.0,2,CLAS,2,...,NaN,1950,Great Britain,1,13 May 1950,Nino Farina,Alfa Romeo,70.0,2:13:23.600,NaN
2,https://www.formula1.com/en/results/1950/races...,3,Reg Parnell,Alfa Romeo,70.0,+52.000s,4.0,3,CLAS,4,...,NaN,1950,Great Britain,1,13 May 1950,Nino Farina,Alfa Romeo,70.0,2:13:23.600,NaN
3,https://www.formula1.com/en/results/1950/races...,4,Yves Giraud-Cabantous,Talbot-Lago,68.0,+2 laps,3.0,4,CLAS,6,...,NaN,1950,Great Britain,1,13 May 1950,Nino Farina,Alfa Romeo,70.0,2:13:23.600,NaN
4,https://www.formula1.com/en/results/1950/races...,5,Louis Rosier,Talbot-Lago,68.0,+2 laps,2.0,5,CLAS,9,...,NaN,1950,Great Britain,1,13 May 1950,Nino Farina,Alfa Romeo,70.0,2:13:23.600,NaN


### Merge races_all_results_pits_winner_fl -- drivers_info on [year, driver]

In [88]:
races_all_results_pits_winner_fl_info = pd.merge(races_all_results_pits_winner_fl, drivers_info, on=["year", "driver"], how="left")
races_all_results_pits_winner_fl_info.shape # Still has 25101 rows

(25101, 23)

In [89]:
races_all_results_pits_winner_fl_info.head()

,race_url,raw_end_position,driver,team_name_x,laps_completed,time_gap,points,end_position,position_status,start_position,...,race,round,date,winner,team_name,laps,total_time,fastest_lap,driver_code,nationality
0,https://www.formula1.com/en/results/1950/races...,1,Nino Farina,Alfa Romeo,70.0,2:13:23.600,9.0,1,CLAS,1,...,Great Britain,1,13 May 1950,Nino Farina,Alfa Romeo,70.0,2:13:23.600,1.0,FAR,ITA
1,https://www.formula1.com/en/results/1950/races...,2,Luigi Fagioli,Alfa Romeo,70.0,+2.600s,6.0,2,CLAS,2,...,Great Britain,1,13 May 1950,Nino Farina,Alfa Romeo,70.0,2:13:23.600,NaN,FAG,ITA
2,https://www.formula1.com/en/results/1950/races...,3,Reg Parnell,Alfa Romeo,70.0,+52.000s,4.0,3,CLAS,4,...,Great Britain,1,13 May 1950,Nino Farina,Alfa Romeo,70.0,2:13:23.600,NaN,PAR,GBR
3,https://www.formula1.com/en/results/1950/races...,4,Yves Giraud-Cabantous,Talbot-Lago,68.0,+2 laps,3.0,4,CLAS,6,...,Great Britain,1,13 May 1950,Nino Farina,Alfa Romeo,70.0,2:13:23.600,NaN,GIR,FRA
4,https://www.formula1.com/en/results/1950/races...,5,Louis Rosier,Talbot-Lago,68.0,+2 laps,2.0,5,CLAS,9,...,Great Britain,1,13 May 1950,Nino Farina,Alfa Romeo,70.0,2:13:23.600,NaN,ROS,FRA


In [90]:
# Remove duplicates
print(races_all_results_pits_winner_fl_info.duplicated(subset=["year", "race", "driver"]).sum())
races_all_results_pits_winner_fl_info = races_all_results_pits_winner_fl_info.drop_duplicates(subset=["year", "race", "driver"])
races_all_results_pits_winner_fl_info.shape


0


(25101, 23)

# Save Rough Merged Dataframe

In [91]:
races_all_results_pits_winner_fl_info.to_csv("../data/intermediate/f1_races_raw.csv", encoding="utf-8", index=False)

# Load Raw Dataframe

In [92]:
f1_races_raw = pd.read_csv("../data/intermediate/f1_races_raw.csv", encoding="utf-8")
f1_races_raw.head()

,race_url,raw_end_position,driver,team_name_x,laps_completed,time_gap,points,end_position,position_status,start_position,...,race,round,date,winner,team_name,laps,total_time,fastest_lap,driver_code,nationality
0,https://www.formula1.com/en/results/1950/races...,1,Nino Farina,Alfa Romeo,70.0,2:13:23.600,9.0,1.0,CLAS,1.0,...,Great Britain,1,13 May 1950,Nino Farina,Alfa Romeo,70.0,2:13:23.600,1.0,FAR,ITA
1,https://www.formula1.com/en/results/1950/races...,2,Luigi Fagioli,Alfa Romeo,70.0,+2.600s,6.0,2.0,CLAS,2.0,...,Great Britain,1,13 May 1950,Nino Farina,Alfa Romeo,70.0,2:13:23.600,NaN,FAG,ITA
2,https://www.formula1.com/en/results/1950/races...,3,Reg Parnell,Alfa Romeo,70.0,+52.000s,4.0,3.0,CLAS,4.0,...,Great Britain,1,13 May 1950,Nino Farina,Alfa Romeo,70.0,2:13:23.600,NaN,PAR,GBR
3,https://www.formula1.com/en/results/1950/races...,4,Yves Giraud-Cabantous,Talbot-Lago,68.0,+2 laps,3.0,4.0,CLAS,6.0,...,Great Britain,1,13 May 1950,Nino Farina,Alfa Romeo,70.0,2:13:23.600,NaN,GIR,FRA
4,https://www.formula1.com/en/results/1950/races...,5,Louis Rosier,Talbot-Lago,68.0,+2 laps,2.0,5.0,CLAS,9.0,...,Great Britain,1,13 May 1950,Nino Farina,Alfa Romeo,70.0,2:13:23.600,NaN,ROS,FRA


# Data Cleaning

### Reorder Columns Logically

In [93]:
f1_races_raw.columns.value_counts().sum()

23

In [94]:
# Sort the columns so they make more sense being next to each other
f1_races_raw = f1_races_raw[["year", "date", "round", "race", "driver", "driver_code", "nationality", "team_name_x", "points", "raw_end_position", "end_position", "position_status", "start_position", "qualifying_time", "time_gap", "laps_completed", "pit_count", "fastest_lap", "winner", "team_name", "laps", "total_time", "race_url"]].reset_index(drop=True)

In [95]:
f1_races_raw.columns.value_counts().sum()

23

### Fix Missing Driver Codes

In [96]:
# Find the rows with missing driver code and fill it in with the first 3 letters of their last name
f1_races_raw.loc[f1_races_raw["driver_code"].isna(), "driver_code"] = f1_races_raw.loc[f1_races_raw["driver_code"].isna(), "driver"].apply(lambda x: x.split()[-1][:3].upper())

In [97]:
f1_races_raw.loc[(pd.isna(f1_races_raw["driver_code"]))]

,year,date,round,race,driver,driver_code,nationality,team_name_x,points,raw_end_position,...,qualifying_time,time_gap,laps_completed,pit_count,fastest_lap,winner,team_name,laps,total_time,race_url


### Insert 0s in fastest_lap

In [98]:
f1_races_raw["fastest_lap"] = f1_races_raw["fastest_lap"].fillna(0)
f1_races_raw["fastest_lap"].unique() # All 0 or 1 now

array([1., 0.])

In [99]:
f1_races_raw.loc[(pd.isna(f1_races_raw["fastest_lap"]))]

,year,date,round,race,driver,driver_code,nationality,team_name_x,points,raw_end_position,...,qualifying_time,time_gap,laps_completed,pit_count,fastest_lap,winner,team_name,laps,total_time,race_url


### Fix team_name Columns

In [100]:
f1_races_raw = f1_races_raw.rename(columns={"team_name_x": "team_name",
                                        "team_name": "winner_team_name"})
f1_races_raw.head(3)

,year,date,round,race,driver,driver_code,nationality,team_name,points,raw_end_position,...,qualifying_time,time_gap,laps_completed,pit_count,fastest_lap,winner,winner_team_name,laps,total_time,race_url
0,1950,13 May 1950,1,Great Britain,Nino Farina,FAR,ITA,Alfa Romeo,9.0,1,...,1:50.800,2:13:23.600,70.0,NaN,1.0,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...
1,1950,13 May 1950,1,Great Britain,Luigi Fagioli,FAG,ITA,Alfa Romeo,6.0,2,...,1:51.000,+2.600s,70.0,NaN,0.0,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...
2,1950,13 May 1950,1,Great Britain,Reg Parnell,PAR,GBR,Alfa Romeo,4.0,3,...,1:52.200,+52.000s,70.0,NaN,0.0,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...


### Consolidate position_status

In [101]:
f1_races_raw["position_status"].value_counts()

position_status
CLAS    15990
NC       8751
DNF       293
DQ         63
DNS         2
EX          2
Name: count, dtype: int64

In [102]:
f1_races_raw["position_status"] = f1_races_raw["position_status"].replace("EX", "DQ")

In [103]:
f1_races_raw["position_status"].value_counts()

position_status
CLAS    15990
NC       8751
DNF       293
DQ         65
DNS         2
Name: count, dtype: int64

### Convert Some Values to NaN

In [104]:
f1_races_raw.isna().sum()

year                    0
date                    0
round                   0
race                    0
driver                  0
driver_code             0
nationality          4276
team_name              39
points                  0
raw_end_position        0
end_position            0
position_status         0
start_position          0
qualifying_time       423
time_gap              379
laps_completed        177
pit_count           14292
fastest_lap             0
winner                  0
winner_team_name        0
laps                    0
total_time              0
race_url                0
dtype: int64

In [105]:
# Odd non-finishing data
'''
time_gap -- DNC, DNF, DNS, DSQ, N/A, OK, SHC

qualifying_time -- Unknown
'''

'\ntime_gap -- DNC, DNF, DNS, DSQ, N/A, OK, SHC\n\nqualifying_time -- Unknown\n'

In [106]:
f1_races_raw["time_gap"] = f1_races_raw["time_gap"].replace(["DNC", "DNF", "DNS", "DSQ", "N/A", "OK"], np.nan)
f1_races_raw["qualifying_time"] = f1_races_raw["qualifying_time"].replace("Unknown", np.nan)

In [107]:
f1_races_raw.isna().sum()

year                    0
date                    0
round                   0
race                    0
driver                  0
driver_code             0
nationality          4276
team_name              39
points                  0
raw_end_position        0
end_position            0
position_status         0
start_position          0
qualifying_time       560
time_gap             9871
laps_completed        177
pit_count           14292
fastest_lap             0
winner                  0
winner_team_name        0
laps                    0
total_time              0
race_url                0
dtype: int64

### Fix Name Spacing Issues

In [108]:
columns_to_fix = ["race", "driver", "team_name", "winner", "winner_team_name"]
for col in columns_to_fix:
    f1_races_raw[col] = f1_races_raw[col].str.replace("  ", " ")

### Impute Missing Nationalities

In [109]:
# Get rows with missing nationality
null_nationality = f1_races_raw[f1_races_raw["nationality"].isna()]

# For each driver with missing nationality, try to find matching driver with known nationality
for idx, row in null_nationality.iterrows():
    driver_name = row["driver"]
    matching_drivers = f1_races_raw[(f1_races_raw["driver"] == driver_name) & (f1_races_raw["nationality"].notna())]
    
    if len(matching_drivers) > 0:
        # Take the first matching nationality found
        imputed_nationality = matching_drivers["nationality"].iloc[0]
        f1_races_raw.loc[idx, "nationality"] = imputed_nationality

# Count how many were fixed
print(f"Imputed: {len(null_nationality) - f1_races_raw["nationality"].isna().sum()}")

Imputed: 2662


### Merge Shared Car Rows

In [110]:
f1_races_raw.shape # 25101

(25101, 23)

In [111]:
# Manual Groups

# 15 groups
manual_groups = {
    (1950, "Great Britain"): [["Peter Walker", "Tony Rolt"]],
    (1950, "Italy"): [["Piero Taruffi", "Juan Manuel Fangio"]],
    (1952, "Switzerland"): [["Andre Simon", "Nino Farina"]],
    (1953, "Belgium"): [["Johnny Claes", "Juan Manuel Fangio"]],
    (1954, "Great Britain"): [
        ["Prince Bira", "Ron Flockhart"],
        ["Luigi Villoresi", "Alberto Ascari"]
    ],
    (1954, "Spain"): [["Toulo de Graffenried", "Ottorino Volonterio"]],
    (1955, "Great Britain"): [["Tony Rolt", "Peter Walker"]],
    (1955, "Argentina"): [
        ["Eugenio Castellotti", "Luigi Villoresi"],
        ["Clemar Bucci", "Carlos Menditeguy"]
    ],
    (1956, "Germany"): [
        ["Alfonso de Portago", "Peter Collins"],
        ["Luigi Villoresi", "Eugenio Castellotti"]
    ],
    (1956, "Italy"): [
        ["Umberto Maglioli", "Jean Behra"],
        ["Luigi Villoresi", "Jo Bonnier"]
    ],
    (1957, "Monaco"): [["Giorgio Scarlatti", "Harry Schell"]]
}

manual_grouped_rows = []
processed_indices = set()

for (year, race), list_of_groups in manual_groups.items():
    for group in list_of_groups:
        group_df = f1_races_raw[
            (f1_races_raw["year"] == year) &
            (f1_races_raw["race"] == race) &
            (f1_races_raw["driver"].isin(group))
        ]

        if set(group).issubset(set(group_df["driver"])):

            if not group_df.empty:
                first_row = group_df.iloc[0]
                race_url = first_row["race_url"]
                end_position = first_row["end_position"]
                team_name = first_row["team_name"]

                match_group = group_df

                if not match_group.empty:
                    combined_driver = " / ".join(match_group["driver"].dropna().astype(str).unique())
                    combined_code = " / ".join(match_group["driver_code"].dropna().astype(str).unique())

                    def prioritize(col):
                        col_vals = match_group[col].dropna()
                        return col_vals.iloc[0] if not col_vals.empty else np.nan

                    # Prioritize times, then + values, then NaN, ignore SHC
                    def prioritize_time_gap(values):
                        time_values = [val for val in values if isinstance(val, str) and ':' in val and '.' in val]
                        if time_values:
                            return time_values[0]
                        plus_values = [val for val in values if isinstance(val, str) and val.startswith("+")]
                        if plus_values:
                            return plus_values[0]
                        nan_values = [val for val in values if pd.isna(val)]
                        if nan_values:
                            return np.nan
                        return np.nan

                    # Combine data with priorities
                    combined_data = {
                        "year": prioritize("year"),
                        "date": prioritize("date"),
                        "round": prioritize("round"),
                        "race": prioritize("race"),
                        "driver": combined_driver,
                        "driver_code": combined_code,
                        "nationality": prioritize("nationality"),
                        "team_name": team_name,
                        "points": prioritize("points"),
                        "raw_end_position": prioritize("raw_end_position"),
                        "end_position": end_position,
                        "position_status": prioritize("position_status"),
                        "start_position": prioritize("start_position"),
                        "qualifying_time": prioritize("qualifying_time"),
                        "time_gap": prioritize_time_gap(match_group["time_gap"].values),
                        "laps_completed": prioritize("laps_completed"),
                        "pit_count": prioritize("pit_count"),
                        "fastest_lap": prioritize("fastest_lap"),
                        "winner": prioritize("winner"),
                        "winner_team_name": prioritize("winner_team_name"),
                        "laps": prioritize("laps"),
                        "total_time": prioritize("total_time"),
                        "race_url": race_url,
                        "is_manually_grouped": True
                    }
                    
                    manual_grouped_rows.append(combined_data)
                    processed_indices.update(match_group.index)

# Greate the groups dataframe
manual_groups_df = pd.DataFrame(manual_grouped_rows)
print(f"Number of manually created groups: {len(manual_groups_df)}")

# Create the original rows dataframe
original_rows_df = f1_races_raw.loc[list(processed_indices)].copy()
print(f"Number of original rows grouped: {len(original_rows_df)}")

Number of manually created groups: 15
Number of original rows grouped: 30


In [112]:
# Manual Individuals

# 18 individuals
excluded_individuals = [
    {"year": 1950, "race": "Great Britain", "driver": "Leslie Johnson"},
    {"year": 1950, "race": "Italy", "driver": "Consalvo Sanesi"},
    {"year": 1952, "race": "Switzerland", "driver": "Louis Rosier"},
    {"year": 1953, "race": "Belgium", "driver": "Jose Froilan Gonzalez"},
    {"year": 1954, "race": "Great Britain", "driver": "Stirling Moss"},
    {"year": 1954, "race": "Great Britain", "driver": "Roy Salvadori"},
    {"year": 1954, "race": "Spain", "driver": "Sergio Mantovani"},
    {"year": 1954, "race": "Spain", "driver": "Harry Schell"},
    {"year": 1954, "race": "Spain", "driver": "Stirling Moss"},
    {"year": 1955, "race": "Argentina", "driver": "Alberto Ascari"},
    {"year": 1955, "race": "Argentina", "driver": "Alberto Uria"},
    {"year": 1955, "race": "Great Britain", "driver": "Kenneth McAlpine"},
    {"year": 1955, "race": "Great Britain", "driver": "Leslie Marr"},
    {"year": 1956, "race": "Germany", "driver": "Giorgio Scarlatti"},
    {"year": 1956, "race": "Italy", "driver": "Bruce Halford"},
    {"year": 1956, "race": "Monaco", "driver": "Robert Manzon"},
    {"year": 1957, "race": "Monaco", "driver": "Carlos Menditeguy"},
    {"year": 1957, "race": "Monaco", "driver": "Horace Gould"},
]

excluded_df = pd.DataFrame(excluded_individuals)

# Merge to get all columns for those specific rows
manual_individuals_df = f1_races_raw.merge(
    excluded_df, 
    on=["year", "race", "driver"], 
    how="inner"
)

print(f"Number of excluded individuals: {len(manual_individuals_df)}")

Number of excluded individuals: 18


In [113]:
# Exclude Rows

# Merge with excluded individuals and manually grouped rows
merged_excluded = f1_races_raw.merge(
    manual_individuals_df[["year", "race", "driver"]],
    on=["year", "race", "driver"],
    how="left",
    indicator="merge_excluded"
)

merged_both = merged_excluded.merge(
    original_rows_df[["year", "race", "driver"]],
    on=["year", "race", "driver"],
    how="left",
    indicator="merge_manual_groups"
)

# Keep only rows that did not match in either dataframe
f1_races_ex = merged_both[(merged_both["merge_excluded"] == "left_only") & (merged_both["merge_manual_groups"] == "left_only")].drop(columns=["merge_excluded", "merge_manual_groups"])

print(f"Original rows: {len(f1_races_raw)}")
print(f"Excluded individual rows ({len(manual_individuals_df)}) + manually grouped rows ({len(original_rows_df)}): {len(f1_races_raw)-len(f1_races_ex)}")
print(f"Remaining rows: {len(f1_races_ex)}")

Original rows: 25101
Excluded individual rows (18) + manually grouped rows (30): 48
Remaining rows: 25053


In [114]:
# Automatic Groups

# Filter for all SHC rows
shc_rows = f1_races_ex[f1_races_ex["time_gap"] == "SHC"]

# Track merged rows
merged_rows = []
processed_indices = set()

# Group by race_url, end_position, and team_name
group_keys = shc_rows.groupby(["race_url", "end_position", "team_name"]).groups

for key, indices in group_keys.items():
    race_url, end_position, team_name = key

    # Find all matching rows (SHC and non-SHC) with same race, position, and team
    match_group = f1_races_ex[
        (f1_races_ex["race_url"] == race_url) &
        (f1_races_ex["end_position"] == end_position) &
        (f1_races_ex["team_name"] == team_name)
    ]

    # Skip if we’ve already processed these
    if set(match_group.index).intersection(processed_indices):
        continue

    # Combine drivers and codes
    combined_driver = " / ".join(match_group["driver"].dropna().astype(str).unique())
    combined_code = " / ".join(match_group["driver_code"].dropna().astype(str).unique())

    # Use non-NA values, preferring ones with actual data
    def prioritize(col):
        col_vals = match_group[col].dropna()
        return col_vals.iloc[0] if not col_vals.empty else np.nan

    def prioritize_time_gap(values):
        # Prefer value that starts with '+', then NaN, ignore SHC
        for val in values:
            if isinstance(val, str) and val.startswith("+"):
                return val
        for val in values:
            if pd.isna(val):
                return np.nan
        return np.nan

    combined_data = {
        "year": prioritize("year"),
        "date": prioritize("date"),
        "round": prioritize("round"),
        "race": prioritize("race"),
        "driver": combined_driver,
        "driver_code": combined_code,
        "nationality": prioritize("nationality"),
        "team_name": team_name,
        "points": prioritize("points"),
        "raw_end_position": prioritize("raw_end_position"),
        "end_position": end_position,
        "position_status": prioritize("position_status"),
        "start_position": prioritize("start_position"),
        "qualifying_time": prioritize("qualifying_time"),
        "time_gap": prioritize_time_gap(match_group["time_gap"].values),
        "laps_completed": prioritize("laps_completed"),
        "pit_count": prioritize("pit_count"),
        "fastest_lap": prioritize("fastest_lap"),
        "winner": prioritize("winner"),
        "winner_team_name": prioritize("winner_team_name"),
        "laps": prioritize("laps"),
        "total_time": prioritize("total_time"),
        "race_url": race_url,
        "is_manually_grouped": True
    }

    merged_rows.append(combined_data)
    processed_indices.update(match_group.index)

# Drop all the original shared car rows and add merged ones
f1_races_ex = f1_races_ex.drop(list(processed_indices))
merged_df = pd.DataFrame(merged_rows)
f1_races_ex = pd.concat([f1_races_ex, merged_df], ignore_index=True)

# Replace raw dataframe with the grouped one
f1_races_raw = f1_races_ex.copy()

# Count how many rows were merged and how many merged entries were added
print(f"Rows merged (dropped): {len(processed_indices)}") # 158
print(f"Merged rows created: {len(merged_rows)}") # 78

Rows merged (dropped): 158
Merged rows created: 78


In [115]:
# Merge back the excluded individuals
f1_races_raw = pd.concat([f1_races_raw, manual_individuals_df], ignore_index=True)

# Merge back the manual groups
f1_races_raw = pd.concat([f1_races_raw, manual_groups_df], ignore_index=True)

# Sort
f1_races_raw.sort_values(by=["year", "round", "end_position"], inplace=True)
f1_races_raw.reset_index(drop=True, inplace=True)

# Verify that all rows were added
print(f"Remaining rows: {len(f1_races_raw)}") # 25077 - 24980 = 97

# Drop the is_manually_grouped column
f1_races_raw = f1_races_raw.drop('is_manually_grouped', axis=1)

Remaining rows: 25006


In [116]:
f1_races_raw[f1_races_raw['driver'].str.count('/') == 2]

,year,date,round,race,driver,driver_code,nationality,team_name,points,raw_end_position,...,qualifying_time,time_gap,laps_completed,pit_count,fastest_lap,winner,winner_team_name,laps,total_time,race_url
538,1953,30 May 1953,2,Indianapolis,Tony Bettenhausen / Chuck Stevenson / Gene Har...,BET / STE / HAR,USA,Kuzma Offenhauser,0.0,9,...,4:24.660,NaN,196.0,NaN,0.0,Bill Vukovich,Kurtis Kraft Offenhauser,200.0,3:53:01.690,https://www.formula1.com/en/results/1953/races...
545,1953,30 May 1953,2,Indianapolis,Rodger Ward / Duke Dinsmore / Andy Linden,WAR / DIN / LIN,USA,Kurtis Kraft Offenhauser,0.0,16,...,4:21.880,NaN,177.0,NaN,0.0,Bill Vukovich,Kurtis Kraft Offenhauser,200.0,3:53:01.690,https://www.formula1.com/en/results/1953/races...
548,1953,30 May 1953,2,Indianapolis,Spider Webb / Jackie Holmes / Johnny Thomson,WEB / HOL / THO,USA,Kurtis Kraft Offenhauser,0.0,19,...,4:24.380,NaN,166.0,NaN,0.0,Bill Vukovich,Kurtis Kraft Offenhauser,200.0,3:53:01.690,https://www.formula1.com/en/results/1953/races...
758,1954,31 May 1954,2,Indianapolis,Jimmy Jackson / Marshall Teague / Tony Bettenh...,JAC / TEA / BET,USA,Kurtis Kraft Offenhauser,0.0,15,...,4:20.350,+4 laps,196.0,NaN,0.0,Bill Vukovich,Kurtis Kraft Offenhauser,200.0,3:49:17.270,https://www.formula1.com/en/results/1954/races...
911,1955,16 Jan 1955,1,Argentina,Jose Froilan Gonzalez / Maurice Trintignant / ...,GON / TRI / FAR,ARG,Ferrari,2.0,2,...,1:43.100,+89.600s,96.0,NaN,0.0,Juan Manuel Fangio,Mercedes-Benz,96.0,3:00:38.600,https://www.formula1.com/en/results/1955/races...
913,1955,16 Jan 1955,1,Argentina,Hans Herrmann / Karl Kling / Stirling Moss,HER / KLI / MOS,GER,Mercedes-Benz,1.0,4,...,1:44.900,+2 laps,94.0,NaN,0.0,Juan Manuel Fangio,Mercedes-Benz,96.0,3:00:38.600,https://www.formula1.com/en/results/1955/races...
1208,1957,13 Jan 1957,1,Argentina,Cesare Perdisa / Wolfgang von Trips / Peter Co...,PER / VON / COL,ITA,Ferrari,0.0,6,...,1:48.600,+2 laps,98.0,NaN,0.0,Juan Manuel Fangio,Maserati,100.0,3:00:55.900,https://www.formula1.com/en/results/1957/races...


In [117]:
f1_races_raw[f1_races_raw['driver'].str.count('/') > 2]

,year,date,round,race,driver,driver_code,nationality,team_name,points,raw_end_position,...,qualifying_time,time_gap,laps_completed,pit_count,fastest_lap,winner,winner_team_name,laps,total_time,race_url
754,1954,31 May 1954,2,Indianapolis,Art Cross / Andy Linden / Jimmy Davies / Johnn...,CRO / LIN / DAV / PAR / HAN,USA,Kurtis Kraft Offenhauser,0.0,11,...,4:19.600,NaN,200.0,NaN,0.0,Bill Vukovich,Kurtis Kraft Offenhauser,200.0,3:49:17.270,https://www.formula1.com/en/results/1954/races...


### Fix Datatypes

In [118]:
f1_races_raw.dtypes

year                  int64
date                 object
round                 int64
race                 object
driver               object
driver_code          object
nationality          object
team_name            object
points              float64
raw_end_position     object
end_position        float64
position_status      object
start_position      float64
qualifying_time      object
time_gap             object
laps_completed      float64
pit_count           float64
fastest_lap         float64
winner               object
winner_team_name     object
laps                float64
total_time           object
race_url             object
dtype: object

In [119]:
# Convert date column to datetime
f1_races_raw['date'] = pd.to_datetime(f1_races_raw['date'], format='%d %b %Y')

# Convert fastest_lap to integer type
f1_races_raw['fastest_lap'] = f1_races_raw['fastest_lap'].fillna(0).astype(int)

# Convert start and end position to numeric
f1_races_raw["start_position"] = pd.to_numeric(f1_races_raw["start_position"], errors="coerce")
f1_races_raw["end_position"] = pd.to_numeric(f1_races_raw["end_position"], errors="coerce")

In [120]:
f1_races_raw.head()

,year,date,round,race,driver,driver_code,nationality,team_name,points,raw_end_position,...,qualifying_time,time_gap,laps_completed,pit_count,fastest_lap,winner,winner_team_name,laps,total_time,race_url
0,1950,1950-05-13,1,Great Britain,Nino Farina,FAR,ITA,Alfa Romeo,9.0,1,...,1:50.800,2:13:23.600,70.0,NaN,1,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...
1,1950,1950-05-13,1,Great Britain,Luigi Fagioli,FAG,ITA,Alfa Romeo,6.0,2,...,1:51.000,+2.600s,70.0,NaN,0,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...
2,1950,1950-05-13,1,Great Britain,Reg Parnell,PAR,GBR,Alfa Romeo,4.0,3,...,1:52.200,+52.000s,70.0,NaN,0,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...
3,1950,1950-05-13,1,Great Britain,Yves Giraud-Cabantous,GIR,FRA,Talbot-Lago,3.0,4,...,1:53.400,+2 laps,68.0,NaN,0,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...
4,1950,1950-05-13,1,Great Britain,Louis Rosier,ROS,FRA,Talbot-Lago,2.0,5,...,1:56.000,+2 laps,68.0,NaN,0,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...


In [121]:
f1_races_raw.dtypes

year                         int64
date                datetime64[ns]
round                        int64
race                        object
driver                      object
driver_code                 object
nationality                 object
team_name                   object
points                     float64
raw_end_position            object
end_position               float64
position_status             object
start_position             float64
qualifying_time             object
time_gap                    object
laps_completed             float64
pit_count                  float64
fastest_lap                  int32
winner                      object
winner_team_name            object
laps                       float64
total_time                  object
race_url                    object
dtype: object

### Fix time_gap Column 

In [122]:
# Remove 's' from time_gap values that end with 's' but not from those containing 'laps'
f1_races_raw.loc[~f1_races_raw["time_gap"].str.contains("laps", na=False), "time_gap"] = \
    f1_races_raw.loc[~f1_races_raw["time_gap"].str.contains("laps", na=False), "time_gap"].str.replace("s$", "", regex=True)

### Add % Season Points Column

In [123]:
### Fix Datatypes
season_total_points = f1_races_raw.groupby("year")["points"].sum().reset_index()
season_total_points = season_total_points.rename(columns={"points": "season_total_points"})

# Merge season_total_points with f1_races_raw
f1_races_raw = f1_races_raw.merge(season_total_points, on="year", how="left")

# Add the new column
f1_races_raw["pct_season_points"] = (f1_races_raw["points"] / f1_races_raw["season_total_points"])

# Drop the temporary season_total_points column
f1_races_raw = f1_races_raw.drop("season_total_points", axis=1)
f1_races_raw.head()

,year,date,round,race,driver,driver_code,nationality,team_name,points,raw_end_position,...,time_gap,laps_completed,pit_count,fastest_lap,winner,winner_team_name,laps,total_time,race_url,pct_season_points
0,1950,1950-05-13,1,Great Britain,Nino Farina,FAR,ITA,Alfa Romeo,9.0,1,...,2:13:23.600,70.0,NaN,1,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...,0.055556
1,1950,1950-05-13,1,Great Britain,Luigi Fagioli,FAG,ITA,Alfa Romeo,6.0,2,...,+2.600,70.0,NaN,0,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...,0.037037
2,1950,1950-05-13,1,Great Britain,Reg Parnell,PAR,GBR,Alfa Romeo,4.0,3,...,+52.000,70.0,NaN,0,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...,0.024691
3,1950,1950-05-13,1,Great Britain,Yves Giraud-Cabantous,GIR,FRA,Talbot-Lago,3.0,4,...,+2 laps,68.0,NaN,0,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...,0.018519
4,1950,1950-05-13,1,Great Britain,Louis Rosier,ROS,FRA,Talbot-Lago,2.0,5,...,+2 laps,68.0,NaN,0,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...,0.012346


In [124]:
print(f1_races_raw[(f1_races_raw["year"] == 2024) & (f1_races_raw["driver"] == "Lando Norris")]["pct_season_points"].sum())
print(f1_races_raw[(f1_races_raw["year"] == 2024) & (f1_races_raw["driver"] == "Max Verstappen")]["pct_season_points"].sum())

0.14081047891936144
0.16332378223495703


In [125]:
f1_races_raw[f1_races_raw["year"] == 2024].groupby("driver")["pct_season_points"].sum().sort_values(ascending=False)

driver
Max Verstappen      0.163324
Lando Norris        0.140810
Charles Leclerc     0.133852
Oscar Piastri       0.108473
Carlos Sainz        0.107245
George Russell      0.092509
Lewis Hamilton      0.084732
Sergio Perez        0.056488
Fernando Alonso     0.028653
Pierre Gasly        0.016373
Nico Hulkenberg     0.014736
Yuki Tsunoda        0.011871
Lance Stroll        0.009824
Esteban Ocon        0.009415
Kevin Magnussen     0.005731
Alexander Albon     0.004912
Oliver Bearman      0.002865
Daniel Ricciardo    0.002865
Franco Colapinto    0.002047
Liam Lawson         0.001637
Zhou Guanyu         0.001637
Logan Sargeant      0.000000
Jack Doohan         0.000000
Valtteri Bottas     0.000000
Name: pct_season_points, dtype: float64

### Add Place Differential Column

In [126]:
# Calculate place differential (negative means gained positions, positive means lost positions)
f1_races_raw["place_differential"] = f1_races_raw["end_position"] - f1_races_raw["start_position"]

### Map Constructors Common Name

In [127]:
f1_races_raw["team_name"].isna().sum()

39

In [128]:
f1_races_raw["team_name"] = f1_races_raw["team_name"].fillna("No Team")
print(f1_races_raw["team_name"].isna().sum())
f1_races_raw.head()

0


,year,date,round,race,driver,driver_code,nationality,team_name,points,raw_end_position,...,laps_completed,pit_count,fastest_lap,winner,winner_team_name,laps,total_time,race_url,pct_season_points,place_differential
0,1950,1950-05-13,1,Great Britain,Nino Farina,FAR,ITA,Alfa Romeo,9.0,1,...,70.0,NaN,1,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...,0.055556,0.0
1,1950,1950-05-13,1,Great Britain,Luigi Fagioli,FAG,ITA,Alfa Romeo,6.0,2,...,70.0,NaN,0,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...,0.037037,0.0
2,1950,1950-05-13,1,Great Britain,Reg Parnell,PAR,GBR,Alfa Romeo,4.0,3,...,70.0,NaN,0,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...,0.024691,-1.0
3,1950,1950-05-13,1,Great Britain,Yves Giraud-Cabantous,GIR,FRA,Talbot-Lago,3.0,4,...,68.0,NaN,0,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...,0.018519,-2.0
4,1950,1950-05-13,1,Great Britain,Louis Rosier,ROS,FRA,Talbot-Lago,2.0,5,...,68.0,NaN,0,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...,0.012346,-4.0


In [129]:
constructor_mapping = {
    # Red Bull
    'Red Bull Racing Renault': 'Red Bull',
    'Red Bull Renault': 'Red Bull',
    'RBR Renault': 'Red Bull',
    'RBR Cosworth': 'Red Bull',
    'RBR Ferrari': 'Red Bull',
    'Red Bull Racing TAG Heuer': 'Red Bull',
    'Red Bull Racing Honda': 'Red Bull',
    'Red Bull Racing RBPT': 'Red Bull',
    'Red Bull Racing Honda RBPT': 'Red Bull',
    
    # AlphaTauri/Toro Rosso
    'Toro Rosso': 'Toro Rosso',
    'STR Ferrari': 'Toro Rosso',
    'STR Renault': 'Toro Rosso',
    'STR Cosworth': 'Toro Rosso',
    'Toro Rosso Ferrari': 'Toro Rosso',
    'Scuderia Toro Rosso Honda': 'Toro Rosso',
    'AlphaTauri Honda': 'AlphaTauri',
    'AlphaTauri RBPT': 'AlphaTauri',
    'AlphaTauri Honda RBPT': 'AlphaTauri',
    
    # Racing Bulls
    'RB Honda RBPT': 'Racing Bulls',
    
    # Ferrari
    'Ferrari': 'Ferrari',
    'Ferrari Jaguar': 'Ferrari',
    'Thin Wall Ferrari': 'Ferrari',
    
    # Mercedes
    'Mercedes': 'Mercedes',
    'Mercedes-Benz': 'Mercedes',
    
    # Aston Martin
    'Aston Martin Mercedes': 'Aston Martin',
    'Aston Martin Aramco Mercedes': 'Aston Martin',
    'Aston Butterworth': 'Aston Martin',
    'Aston Martin': 'Aston Martin',
    
    # McLaren
    'McLaren Ford': 'McLaren',
    'McLaren TAG': 'McLaren',
    'McLaren Honda': 'McLaren',
    'McLaren Peugeot': 'McLaren',
    'McLaren Renault': 'McLaren',
    'McLaren BRM': 'McLaren',
    'McLaren Mercedes': 'McLaren',
    'McLaren Serenissima': 'McLaren',
    'Mclaren BRM': 'McLaren',
    'McLaren Alfa Romeo': 'McLaren',
    
    # Williams
    'Williams Ford': 'Williams',
    'Williams Renault': 'Williams',
    'Williams Honda': 'Williams',
    'Williams Judd': 'Williams',
    'Williams BMW': 'Williams',
    'Williams Toyota': 'Williams',
    'Williams Cosworth': 'Williams',
    'Williams Mecachrome': 'Williams',
    'Williams Supertec': 'Williams',
    'Williams Mercedes': 'Williams',
    'Frank Williams Racing Cars/Williams': 'Williams',
    
    # Renault
    'Renault': 'Renault',

    # Alpine
    'Alpine Renault': 'Alpine',
    
    # Lotus
    'Lotus Renault': 'Lotus',
    'Lotus Ford': 'Lotus',
    'Lotus Climax': 'Lotus',
    'Lotus BRM': 'Lotus',
    'Lotus Honda': 'Lotus',
    'Lotus Judd': 'Lotus',
    'Lotus Lamborghini': 'Lotus',
    'Lotus Mugen Honda': 'Lotus',
    'Lotus Mercedes': 'Lotus',
    'Lotus Cosworth': 'Lotus',
    'Lotus Maserati': 'Lotus',
    'Lotus Pratt & Whitney': 'Lotus',
    
    # Force India
    'Force India Ferrari': 'Force India',
    'Force India Mercedes': 'Force India',

    # Racing Point
    'Racing Point BWT Mercedes': 'Racing Point',

    # Sauber
    'Sauber': 'Sauber',
    'Sauber Ferrari': 'Sauber',
    'Sauber Petronas': 'Sauber',
    'Sauber BMW': 'Sauber',
    'Sauber Mercedes': 'Sauber',
    'Sauber Ford': 'Sauber',
    'Kick Sauber Ferrari': 'Sauber',

    # Alfa Romeo
    'Alfa Romeo Racing Ferrari': 'Alfa Romeo',
    'Alfa Romeo Ferrari': 'Alfa Romeo',
    'Alfa Romeo': 'Alfa Romeo',
    
    # Haas
    'Haas Ferrari': 'Haas',
    
    # Jordan
    'Jordan Ford': 'Jordan',
    'Jordan Peugeot': 'Jordan',
    'Jordan Hart': 'Jordan',
    'Jordan Honda': 'Jordan',
    'Jordan Yamaha': 'Jordan',
    'Jordan Toyota': 'Jordan',
    'Jordan Mugen Honda': 'Jordan',
    
    # BAR
    'BAR Honda': 'BAR',
    'BAR Supertec': 'BAR',
    
    # Honda
    'Honda': 'Honda',
    
    # Benetton
    'Benetton Ford': 'Benetton',
    'Benetton BMW': 'Benetton',
    'Benetton Renault': 'Benetton',
    'Benetton Playlife': 'Benetton',
    
    # Toyota
    'Toyota': 'Toyota',
    
    # Jaguar
    'Jaguar Cosworth': 'Jaguar',
    
    # Stewart
    'Stewart Ford': 'Stewart',
    
    # BRM
    'BRM': 'BRM',
    'BRM Climax': 'BRM',

    # JBW
    'JBW Maserati': 'JBW',
    'JBW Climax': 'JBW',
    
    # Cooper
    'Cooper Climax': 'Cooper',
    'Cooper Maserati': 'Cooper',
    'Cooper Bristol': 'Cooper',
    'Cooper Castellotti': 'Cooper',
    'Cooper BRM': 'Cooper',
    'Cooper JAP': 'Cooper',
    'Cooper Alta': 'Cooper',
    'Cooper Borgward': 'Cooper',
    'Cooper Alfa Romeo': 'Cooper',
    'Cooper Ferrari': 'Cooper',
    'Cooper ATS': 'Cooper',
    'Cooper Ford': 'Cooper',
    'Cooper OSCA': 'Cooper',
    
    # Brabham
    'Brabham Climax': 'Brabham',
    'Brabham Repco': 'Brabham',
    'Brabham Ford': 'Brabham',
    'Brabham Alfa Romeo': 'Brabham',
    'Brabham BMW': 'Brabham',
    'Brabham BRM': 'Brabham',
    'Brabham Judd': 'Brabham',
    'Brabham Yamaha': 'Brabham',
    
    # Maserati
    'Maserati': 'Maserati',
    'Maserati Offenhauser': 'Maserati',
    'Maserati Milano': 'Maserati',
    'Maserati-Offenhauser': 'Maserati',
    'Maserati OSCA': 'Maserati',
    'Maserati Plate': 'Maserati',
    
    # Ligier
    'Ligier Matra': 'Ligier',
    'Ligier Ford': 'Ligier',
    'Ligier Renault': 'Ligier',
    'Ligier Megatron': 'Ligier',
    'Ligier Mugen Honda': 'Ligier',
    
    # Tyrrell
    'Tyrrell Ford': 'Tyrrell',
    'Tyrrell Renault': 'Tyrrell',
    'Tyrrell Honda': 'Tyrrell',
    'Tyrrell Yamaha': 'Tyrrell',
    'Tyrrell Ilmor': 'Tyrrell',
    
    # Arrows/Footwork
    'Arrows Ford': 'Arrows',
    'Arrows BMW': 'Arrows',
    'Arrows Megatron': 'Arrows',
    'Arrows Yamaha': 'Arrows',
    'Arrows Supertec': 'Arrows',
    'Arrows Asiatech': 'Arrows',
    'Arrows Cosworth': 'Arrows',
    'Arrows': 'Arrows',
    'Footwork Ford': 'Footwork',
    'Footwork Hart': 'Footwork',
    'Footwork Mugen Honda': 'Footwork',
    'Footwork Porsche': 'Footwork',
    
    # Vanwall
    'Vanwall': 'Vanwall',
    
    # Wolf
    'Wolf Ford': 'Wolf',
    'Wolf-Williams': 'Wolf',
    
    # Lola
    'Lola Ford': 'Lola',
    'Lola Lamborghini': 'Lola',
    'Lola Climax': 'Lola',
    'Lola BMW': 'Lola',
    'Lola Hart': 'Lola',
    'Lola Ferrari': 'Lola',

    # March
    'March Ford': 'March',
    'March Judd': 'March',
    'March Ilmor': 'March',
    'March Alfa Romeo': 'March',

    # Minardi
    'Minardi Ford': 'Minardi',
    'Minardi Ferrari': 'Minardi',
    'Minardi Lamborghini': 'Minardi',
    'Minardi Asiatech': 'Minardi',
    'Minardi Cosworth': 'Minardi',
    'Minardi Fondmetal': 'Minardi',
    'Minardi European': 'Minardi',
    'Minardi Hart': 'Minardi',
    'Minardi Motori Moderni': 'Minardi',
    
    # LDS
    'LDS Alfa Romeo': 'LDS',
    'LDS Climax': 'LDS',
    'LDS Repco': 'LDS',

    # Porche
    'Porsche (F2)': 'Porsche',
    'Porsche': 'Porsche',
    'Behra-Porsche': 'Porsche',

    # Scirocco
    'Scirocco BRM': 'Scirocco',
    'Scirocco Climax': 'Scirocco',

    # AFM
    'AFM Kuchen': 'AFM',
    'AFM BMW': 'AFM',
    'AFM Bristol': 'AFM',

    # ATS
    'ATS Ford': 'ATS',
    'ATS': 'ATS',
    'ATS BMW': 'ATS',
    'Derrington-Francis ATS': 'ATS',

    # Leyton House
    'Leyton House Judd': 'Leyton House',
    'Leyton House Ilmor': 'Leyton House',

    # Prost
    'Prost Mugen Honda': 'Prost',
    'Prost Peugeot': 'Prost',
    'Prost Acer': 'Prost',

    # Dallara
    'Dallara Judd': 'Dallara',
    'Dallara Ferrari': 'Dallara',
    'Dallara Ford': 'Dallara',

    # Larrousse
    'Larrousse Lamborghini': 'Larrousse',
    'Larrousse Ford': 'Larrousse',

    # Osella
    'Osella Ford': 'Osella',
    'Osella Alfa Romeo': 'Osella',
    'Osella': 'Osella',
    'Osella Hart': 'Osella',

    # Kurtis Kraft
    'Kurtis Kraft Offenhauser': 'Kurtis Kraft',
    'Kurtis Kraft Novi': 'Kurtis Kraft',
    'Kurtis Kraft Cummins': 'Kurtis Kraft',

    # Marussia
    'Marussia Cosworth': 'Marussia',
    'Marussia Ferrari': 'Marussia',

    # Gordini
    'Simca-Gordini': 'Gordini',
    'Gordini': 'Gordini',

    # Connaught
    'Connaught Lea Francis': 'Connaught',
    'Connaught Alta': 'Connaught',

    # Eagle
    'Eagle Climax': 'Eagle',
    'Eagle Weslake': 'Eagle',

    # RAM
    'RAM Ford': 'RAM',
    'RAM Hart': 'RAM',

    # Shadow
    'Shadow Ford': 'Shadow',
    'Shadow Matra': 'Shadow',

    # Matra
    'Matra Ford': 'Matra',
    'Matra': 'Matra',
    'Matra Cosworth': 'Matra',
    'Matra BRM': 'Matra',

    # ERA
    'ERA': 'ERA',
    'ERA Bristol': 'ERA',

    # Spirit
    'Spirit Honda': 'Spirit',   
    'Spirit Hart': 'Spirit',

    # Frazer Nash
    'Frazer Nash': 'Frazer Nash',
    'Frazer Nash Bristol': 'Frazer Nash',

    # Emeryson
    'Emeryson Alta': 'Emeryson',
    'Emeryson Climax': 'Emeryson',

    # De Tomaso
    'De Tomaso OSCA': 'De Tomaso',
    'De Tomaso Alfa Romeo': 'De Tomaso',
    'De Tomaso Ford': 'De Tomaso',

    # Gilby
    'Gilby Climax': 'Gilby',
    'Gilby BRM': 'Gilby',

    # Tecno
    'Tecno': 'Tecno',
    'Tecno Cosworth': 'Tecno',

    # Ligier
    'Ligier Judd': 'Ligier',
    'Ligier Lamborghini': 'Ligier',

    # Euro Brun
    'Euro Brun Judd': 'Euro Brun',
    'Euro Brun Ford': 'Euro Brun',


    # Other
    'No Team': 'Privateer',
    'Toleman Hart': 'Toleman',       
    'Venturi Lamborghini': 'Venturi',        
    'Onyx Ford': 'Onyx',
    'AGS Ford': 'AGS',   
    'Rial Ford': 'Rial',
    'Zakspeed': 'Zakspeed',
    'Theodore Ford': 'Theodore',
    'Deidt Offenhauser': 'Deidt',
    'Sherman Offenhauser': 'Sherman',
    'Schroeder Offenhauser': 'Schroeder',
    'Kuzma Offenhauser': 'Kuzma',
    'Lesovsky Offenhauser': 'Lesovsky',
    'Watson Offenhauser': 'Watson',
    'Phillips Offenhauser': 'Phillips',
    'Epperly Offenhauser': 'Epperly',
    'Trevis Offenhauser': 'Trevis',
    'HRT Cosworth': 'HRT',
    'Virgin Cosworth': 'Virgin',
    'Caterham Renault': 'Caterham',
    'Milano Speluzzi': 'Milano',
    'Turner Offenhauser': 'Turner',
    'Alta': 'Alta',    
    'Moore Offenhauser': 'Moore',
    'Nichels Offenhauser': 'Nichels',
    'Marchese Offenhauser': 'Marchese',
    'Stevens Offenhauser': 'Stevens',
    'Langley Offenhauser': 'Langley',
    'Ewing Offenhauser': 'Ewing',   
    'Rae Offenhauser': 'Rae',
    'Olson Offenhauser': 'Olson',
    'Wetteroth Offerhauser': 'Wetteroth',
    'Snowberger Offenhauser': 'Snowberger',
    'Adams Offenhauser': 'Adams',
    'HWM Alta': 'HWM',    
    'Lancia': 'Lancia',
    'Talbot-Lago': 'Talbot-Lago',
    'BRP BRM': 'BRP',
    'Hesketh Ford': 'Hesketh',
    'Hill Ford': 'Hill',
    'Ensign Ford': 'Ensign',
    'Penske Ford': 'Penske',
    'Fittipaldi Ford': 'Fittipaldi',
    'ISO Marlboro Ford': 'ISO Marlboro',
    'Iso Marlboro Ford': 'ISO Marlboro',
    'Surtees Ford': 'Surtees',
    'Parnelli Ford': 'Parnelli',
    'Super Aguri Honda': 'Super Aguri',
    'MRT Mercedes': 'Manor',
    'Brawn Mercedes': 'Brawn',
    'Spyker Ferrari': 'Spyker',
    'MF1 Toyota': 'Midland',
    'Veritas': 'Veritas',
    'Pawl Offenhauser': 'Pawl',
    'Hall Offenhauser': 'Hall',
    'Bromme Offenhauser': 'Bromme',
    'OSCA': 'OSCA',
    'BMW': 'BMW',
    'EMW': 'EMW',
    'Pankratz Offenhauser': 'Pankratz',
    'Bugatti': 'Bugatti',
    'Klenk BMW': 'Klenk',
    'Dunn Offenhauser': 'Dunn',    
    'Elder Offenhauser': 'Elder',
    'Christensen Offenhauser': 'Christensen',
    'Sutton Offenhauser': 'Sutton',
    'Tec-Mec Maserati': 'Tec-Mec',
    'Meskowski Offenhauser': 'Meskowski',
    'Scarab': 'Scarab',
    'Ferguson Climax': 'Ferguson',
    'ENB Maserati': 'ENB',
    'Stebro Ford': 'Stebro',               
    'Shannon Climax': 'Shannon',     
    'Protos Cosworth': 'Protos',   
    'Bellasi Ford': 'Bellasi',       
    'Eifelland Ford': 'Eifelland',
    'Politoys Ford': 'Politoys',
    'Connew Ford': 'Connew',
    'Trojan Ford': 'Trojan',
    'Amon Ford': 'Amon',
    'Token Ford': 'Token',
    'Lyncar Ford': 'Lyncar',
    'Boro Ford': 'Boro',
    'Kojima Ford': 'Kojima',
    'LEC Ford': 'LEC',
    'Merzario Ford': 'Merzario',
    'Martini Ford': 'Martini',
    'Rebaque Ford': 'Rebaque',
    'AGS Motori Moderni': 'AGS',
    'Coloni Ford': 'Coloni',
    'Zakspeed Yamaha': 'Zakspeed',
    'Fondmetal Ford': 'Fondmetal',
    'Moda Judd': 'Moda',    
    'Simtek Ford': 'Simtek',
    'Pacific Ilmor': 'Pacific',
    'Forti Ford': 'Forti',
    'Lambo Lamborghini': 'Modena'
}

In [130]:
# Create a new column with mapped constructor names
f1_races_raw["team_name_common"] = f1_races_raw["team_name"].map(constructor_mapping)
f1_races_raw["winner_team_name_common"] = f1_races_raw["winner_team_name"].map(constructor_mapping)

# Check if there are any unmapped constructors
unmapped = f1_races_raw[f1_races_raw["team_name_common"].isna()]["team_name"].unique()
if len(unmapped) > 0:
    print("These constructors were not mapped:", unmapped) # No unmapped anymore

unmapped = f1_races_raw[f1_races_raw["winner_team_name_common"].isna()]["winner_team_name"].unique()
if len(unmapped) > 0:
    print("These winning constructors were not mapped:", unmapped)

f1_races_raw.head()

,year,date,round,race,driver,driver_code,nationality,team_name,points,raw_end_position,...,fastest_lap,winner,winner_team_name,laps,total_time,race_url,pct_season_points,place_differential,team_name_common,winner_team_name_common
0,1950,1950-05-13,1,Great Britain,Nino Farina,FAR,ITA,Alfa Romeo,9.0,1,...,1,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...,0.055556,0.0,Alfa Romeo,Alfa Romeo
1,1950,1950-05-13,1,Great Britain,Luigi Fagioli,FAG,ITA,Alfa Romeo,6.0,2,...,0,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...,0.037037,0.0,Alfa Romeo,Alfa Romeo
2,1950,1950-05-13,1,Great Britain,Reg Parnell,PAR,GBR,Alfa Romeo,4.0,3,...,0,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...,0.024691,-1.0,Alfa Romeo,Alfa Romeo
3,1950,1950-05-13,1,Great Britain,Yves Giraud-Cabantous,GIR,FRA,Talbot-Lago,3.0,4,...,0,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...,0.018519,-2.0,Talbot-Lago,Alfa Romeo
4,1950,1950-05-13,1,Great Britain,Louis Rosier,ROS,FRA,Talbot-Lago,2.0,5,...,0,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...,0.012346,-4.0,Talbot-Lago,Alfa Romeo


In [131]:
f1_races_raw = f1_races_raw.drop(columns=["team_name", "winner_team_name"]).rename(columns={"team_name_common": "team_name", "winner_team_name_common": "winner_team_name"})
f1_races_raw.head()

,year,date,round,race,driver,driver_code,nationality,points,raw_end_position,end_position,...,pit_count,fastest_lap,winner,laps,total_time,race_url,pct_season_points,place_differential,team_name,winner_team_name
0,1950,1950-05-13,1,Great Britain,Nino Farina,FAR,ITA,9.0,1,1.0,...,NaN,1,Nino Farina,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...,0.055556,0.0,Alfa Romeo,Alfa Romeo
1,1950,1950-05-13,1,Great Britain,Luigi Fagioli,FAG,ITA,6.0,2,2.0,...,NaN,0,Nino Farina,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...,0.037037,0.0,Alfa Romeo,Alfa Romeo
2,1950,1950-05-13,1,Great Britain,Reg Parnell,PAR,GBR,4.0,3,3.0,...,NaN,0,Nino Farina,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...,0.024691,-1.0,Alfa Romeo,Alfa Romeo
3,1950,1950-05-13,1,Great Britain,Yves Giraud-Cabantous,GIR,FRA,3.0,4,4.0,...,NaN,0,Nino Farina,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...,0.018519,-2.0,Talbot-Lago,Alfa Romeo
4,1950,1950-05-13,1,Great Britain,Louis Rosier,ROS,FRA,2.0,5,5.0,...,NaN,0,Nino Farina,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...,0.012346,-4.0,Talbot-Lago,Alfa Romeo


### Map Drivers Continent

In [132]:
continents = {
    # Europe
    'GBR': 'Europe',  # United Kingdom
    'FRA': 'Europe',  # France
    'ITA': 'Europe',  # Italy
    'GER': 'Europe',  # Germany
    'ESP': 'Europe',  # Spain
    'BEL': 'Europe',  # Belgium
    'NED': 'Europe',  # Netherlands
    'SUI': 'Europe',  # Switzerland
    'AUT': 'Europe',  # Austria
    'SWE': 'Europe',  # Sweden
    'FIN': 'Europe',  # Finland
    'HUN': 'Europe',  # Hungary
    'POL': 'Europe',  # Poland
    'MON': 'Europe',  # Monaco
    'IRL': 'Europe',  # Ireland
    'POR': 'Europe',  # Portugal
    'DEN': 'Europe',  # Denmark
    'RUS': 'Europe',  # Russia
    'RAF': 'Europe',  # Rhodesian Air Force (competed under British racing license)
    
    # North America
    'USA': 'North America',  # United States
    'CAN': 'North America',  # Canada
    'MEX': 'North America',  # Mexico
    
    # South America
    'BRA': 'South America',  # Brazil
    'ARG': 'South America',  # Argentina
    'VEN': 'South America',  # Venezuela
    'COL': 'South America',  # Colombia
    'CHI': 'South America',  # Chile
    
    # Asia
    'JPN': 'Asia',    # Japan
    'THA': 'Asia',    # Thailand
    'MAS': 'Asia',    # Malaysia
    'IND': 'Asia',    # India
    'INA': 'Asia',    # Indonesia
    'CHN': 'Asia',    # China
    
    # Oceania
    'AUS': 'Oceania', # Australia
    'NZL': 'Oceania', # New Zealand
    
    # Africa
    'RSA': 'Africa',  # South Africa
    'RHO': 'Africa',  # Rhodesia (now Zimbabwe)
}

In [133]:
# Add continent column
f1_races_raw["continent"] = f1_races_raw["nationality"].map(continents)

# Check if there are any unmapped nationalities
unmapped = f1_races_raw[f1_races_raw["continent"].isna()]["nationality"].unique()
if len(unmapped) > 0:
    print("These nationalities were not mapped:", unmapped)

f1_races_raw.head()

These nationalities were not mapped: [nan]


,year,date,round,race,driver,driver_code,nationality,points,raw_end_position,end_position,...,fastest_lap,winner,laps,total_time,race_url,pct_season_points,place_differential,team_name,winner_team_name,continent
0,1950,1950-05-13,1,Great Britain,Nino Farina,FAR,ITA,9.0,1,1.0,...,1,Nino Farina,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...,0.055556,0.0,Alfa Romeo,Alfa Romeo,Europe
1,1950,1950-05-13,1,Great Britain,Luigi Fagioli,FAG,ITA,6.0,2,2.0,...,0,Nino Farina,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...,0.037037,0.0,Alfa Romeo,Alfa Romeo,Europe
2,1950,1950-05-13,1,Great Britain,Reg Parnell,PAR,GBR,4.0,3,3.0,...,0,Nino Farina,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...,0.024691,-1.0,Alfa Romeo,Alfa Romeo,Europe
3,1950,1950-05-13,1,Great Britain,Yves Giraud-Cabantous,GIR,FRA,3.0,4,4.0,...,0,Nino Farina,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...,0.018519,-2.0,Talbot-Lago,Alfa Romeo,Europe
4,1950,1950-05-13,1,Great Britain,Louis Rosier,ROS,FRA,2.0,5,5.0,...,0,Nino Farina,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...,0.012346,-4.0,Talbot-Lago,Alfa Romeo,Europe


### Reorder Columns Again

In [134]:
print(len(f1_races_raw.columns))
f1_races_raw = f1_races_raw[["year", "date", "round", "race", "driver", "driver_code", "nationality", "continent", "team_name", "points", "pct_season_points", "raw_end_position", "end_position", "position_status", "start_position", "place_differential", "qualifying_time", "time_gap", "laps_completed", "pit_count", "fastest_lap", "winner", "winner_team_name", "laps", "total_time", "race_url"]].reset_index(drop=True)
print(len(f1_races_raw.columns))
f1_races_raw.head(20)

26
26


,year,date,round,race,driver,driver_code,nationality,continent,team_name,points,...,qualifying_time,time_gap,laps_completed,pit_count,fastest_lap,winner,winner_team_name,laps,total_time,race_url
0,1950,1950-05-13,1,Great Britain,Nino Farina,FAR,ITA,Europe,Alfa Romeo,9.0,...,1:50.800,2:13:23.600,70.0,NaN,1,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...
1,1950,1950-05-13,1,Great Britain,Luigi Fagioli,FAG,ITA,Europe,Alfa Romeo,6.0,...,1:51.000,+2.600,70.0,NaN,0,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...
2,1950,1950-05-13,1,Great Britain,Reg Parnell,PAR,GBR,Europe,Alfa Romeo,4.0,...,1:52.200,+52.000,70.0,NaN,0,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...
3,1950,1950-05-13,1,Great Britain,Yves Giraud-Cabantous,GIR,FRA,Europe,Talbot-Lago,3.0,...,1:53.400,+2 laps,68.0,NaN,0,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...
4,1950,1950-05-13,1,Great Britain,Louis Rosier,ROS,FRA,Europe,Talbot-Lago,2.0,...,1:56.000,+2 laps,68.0,NaN,0,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...
5,1950,1950-05-13,1,Great Britain,Bob Gerard,GER,NaN,NaN,ERA,0.0,...,1:57.400,+3 laps,67.0,NaN,0,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...
6,1950,1950-05-13,1,Great Britain,Cuth Harrison,HAR,NaN,NaN,ERA,0.0,...,1:58.400,+3 laps,67.0,NaN,0,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...
7,1950,1950-05-13,1,Great Britain,Philippe Etancelin,ETA,FRA,Europe,Talbot-Lago,0.0,...,1:57.800,+5 laps,65.0,NaN,0,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...
8,1950,1950-05-13,1,Great Britain,David Hampshire,HAM,NaN,NaN,Maserati,0.0,...,2:01.000,+6 laps,64.0,NaN,0,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...
9,1950,1950-05-13,1,Great Britain,Joe Fry / Brian Shawe-Taylor,FRY / SHA,NaN,NaN,Maserati,0.0,...,2:07.000,+6 laps,64.0,NaN,0,Nino Farina,Alfa Romeo,70.0,2:13:23.600,https://www.formula1.com/en/results/1950/races...


# Save Cleaned Dataframe

In [135]:
f1_races_raw.to_csv("../data/final/f1_races_clean.csv", encoding="utf-8", index=False)